# Joining Tables with `dfply`

In [1]:
import pandas as pd
from dfply import *

## Hiding stack traceback

We hide the exception traceback for didactic reasons (code source: [see this post](https://stackoverflow.com/questions/46222753/how-do-i-suppress-tracebacks-in-jupyter)).  Don't run this cell if you want to see a full traceback.

In [2]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

## Example

The following tables were taken from the Wikipedia page for joins and will provide a small but sufficiently complicated example for introducing joins.

In [3]:
dept = pd.read_csv("./data/department.csv")
dept

,DeptID,DeptName
0,31,Sales
1,33,Engineering
2,34,Clerical
3,35,Marketing


In [4]:
empl = pd.read_csv("./data/employee.csv")
empl

,LastName,DeptID
0,Rafferty,31.0
1,Jones,33.0
2,Heisenberg,33.0
3,Robinson,34.0
4,Smith,34.0
5,Williams,NaN


## Left Outer Join

Use `left_table >> dfply.left_join(right_table, by=common_col_name)`

In [5]:
empl >> left_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,Williams,NaN,NaN


## Rename when the column names are different

In [6]:
empl2 = empl.fillna(0) >> rename(deptIDL = 'DeptID') 
empl2

,LastName,deptIDL
0,Rafferty,31.0
1,Jones,33.0
2,Heisenberg,33.0
3,Robinson,34.0
4,Smith,34.0
5,Williams,0.0


In [7]:
empl2 >> left_join(dept, by='DeptID')

KeyError: 'DeptID'

In [8]:
empl2 >> rename(DeptID = 'deptIDL') >> left_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,Williams,0.0,NaN


## Right Outer Join

Use `right_table >> dfply.right_join(left_table, by=common_col_name)`

In [9]:
empl >> right_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,NaN,35.0,Marketing


## Full Outer Join

Use `left_table >> dfply.outer_join(right_table, by=common_col_name)`

In [10]:
empl >> outer_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,Williams,NaN,NaN
6,NaN,35.0,Marketing


## Inner Join

Use `left_table >> dfply.inner_join(right_table, by=common_col_name)`

In [11]:
empl >> inner_join(dept, by='DeptID')

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical


## Example 2 - Joining Batting to People

As an example of a joining on more than 1 column, consider the following task.

**Task:** Create a table with the total runs score for each team in 2009.  Include the teams proper name and the name of their home park.

In [12]:
files = ("./data/baseball/core/Batting.csv", 
         "./data/baseball/core/Teams.csv")

In [13]:

batting, teams = [pd.read_csv(f) for f in files]

## Things we need from `Batting.csv`

* Year
* Runs
* Team

In [14]:
batting.head(2)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0


#### Selecting down to the necessary columns

In [15]:
batting_select = (batting
                  >> select(X.teamID,
                            X.yearID,
                            X.R)
                 )
batting_select.head()

,teamID,yearID,R
0,TRO,1871,0
1,RC1,1871,30
2,CL1,1871,28
3,WS3,1871,28
4,RC1,1871,29


In [16]:
batting_select.shape

(107429, 3)

## Things we need from `Teams.csv`

* Year
* Runs
* Team

In [17]:
teams.head(2)

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,...,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1


#### Selecting down to the necessary columns

In [18]:
teams_select = (teams >> select(X.yearID,
                                X.teamID,
                                X.name,
                                X.park))
teams_select.head()

,yearID,teamID,name,park
0,1871,BS1,Boston Red Stockings,South End Grounds I
1,1871,CH1,Chicago White Stockings,Union Base-Ball Grounds
2,1871,CL1,Cleveland Forest Citys,National Association Grounds
3,1871,FW1,Fort Wayne Kekiongas,Hamilton Field
4,1871,NY2,New York Mutuals,Union Grounds (Brooklyn)


In [19]:
teams_select.shape

(2925, 4)

## How can we match these columns?

We will need to match on both `teamID` and `yearID` because

1. Team names might change
2. The teams park depends on the year.

## Determining the correct join.

Facts of the task:

* We want batting statistics for all players in 2009
* We want only teams names and parks for teams that played that year.

**Question:** Which join? 
**Answer:** `batting >> left_join(teams)` on both year and 

## How to join on multiple columns

Pass a list of columns names to `by`

In [20]:
df_joined = batting_select >> left_join(teams_select, by=['teamID', 'yearID'])
df_joined.head()

,teamID,yearID,R,name,park
0,TRO,1871,0,Troy Haymakers,Haymakers' Grounds
1,RC1,1871,30,Rockford Forest Citys,Agricultural Society Fair Grounds
2,CL1,1871,28,Cleveland Forest Citys,National Association Grounds
3,WS3,1871,28,Washington Olympics,Olympics Grounds
4,RC1,1871,29,Rockford Forest Citys,Agricultural Society Fair Grounds


#### Filter, group by, and aggregate to get the desired result.

In [21]:
runs_per_team09 = (df_joined
                   >> filter_by(X.yearID == 2009)
                   >> group_by(X.name, X.park)
                   >> summarise(total_runs = X.R.sum()))
runs_per_team09.head()

,park,name,total_runs
0,Chase Field,Arizona Diamondbacks,720
1,Turner Field,Atlanta Braves,735
2,Oriole Park at Camden Yards,Baltimore Orioles,741
3,Fenway Park II,Boston Red Sox,872
4,Wrigley Field,Chicago Cubs,707


In [22]:
runs_per_team09.shape

(30, 3)

## Joins are expensive!

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `inner_join` performs `min(len(A), len(B))` comparisons
* `outer_join` performs `len(A)*len(B)` row comparisons.
* `left_join` and `right_join` are somewhere inbetween (depending on how we treat multiples).

## Outer joins (can) produce LARGE tables.

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `outer_join` might produce `len(A)*len(B)` rows (worst case)
* `left_join` and `right_join` are similar if we don't drop multiples.

#### Last example 

In [23]:
len(batting_select)*len(teams_select)

314229825

## The joining mantra: Filter/aggregate THEN join

By filtering and/or aggregating before joining, you will

* reduce the amount of work
* protect against really large output

#### Filter and aggregate to total runs per team per year

In [24]:
runs = (batting_select
        >> filter_by(X.yearID == 2009)
        >> group_by(X.teamID)
        >> summarise(total_runs = X.R.sum()))

runs.head()

,teamID,total_runs
0,ARI,720
1,ATL,735
2,BAL,741
3,BOS,872
4,CHA,724


#### Filter down to 2009 teams

In [25]:
teams09 = (teams_select
           >> filter_by(X.yearID == 2009)
           >> drop(X.yearID)
          )
teams09.head()

,teamID,name,park
2595,ARI,Arizona Diamondbacks,Chase Field
2596,ATL,Atlanta Braves,Turner Field
2597,BAL,Baltimore Orioles,Oriole Park at Camden Yards
2598,BOS,Boston Red Sox,Fenway Park II
2599,CHA,Chicago White Sox,U.S. Cellular Field


In [26]:
len(runs)*len(teams09)

900

In [27]:
runs_per_team09_new = (runs 
                       >> left_join(teams09, by='teamID')
                       >> drop(X.teamID))
runs_per_team09_new.head()

,total_runs,name,park
0,720,Arizona Diamondbacks,Chase Field
1,735,Atlanta Braves,Turner Field
2,741,Baltimore Orioles,Oriole Park at Camden Yards
3,872,Boston Red Sox,Fenway Park II
4,724,Chicago White Sox,U.S. Cellular Field


In [28]:
runs_per_team09_new.shape

(30, 3)

## Don't worry about speed until it matters!

* I don't emphasize speed most of the time.
* In this case, either approach was nearly instant.
* Still ... `join`s are *so dangerous* that I make an exception here!

## <font color="red"> Exercise 1 </font>

Determine all the players that have hit more than 100 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the files listed below.  To get credit for this exercise, use the join methods presented above.

In [3]:
files = ("./data/baseball/core/Batting.csv", 
         "./data/baseball/core/People.csv",
         "./data/baseball/core/Teams.csv")

In [4]:
batting = pd.read_csv("./data/baseball/core/Batting.csv")
batting.head(2)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0


In [5]:
people = pd.read_csv("./data/baseball/core/People.csv")
people.head(2)

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01


In [23]:
teams = pd.read_csv("./data/baseball/core/Teams.csv")
teams.head(20)

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,...,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1
2,1871,NaN,CL1,CFC,NaN,8,29,NaN,10,19,...,15,0.818,Cleveland Forest Citys,National Association Grounds,NaN,96,100,CLE,CL1,CL1
3,1871,NaN,FW1,KEK,NaN,7,19,NaN,7,12,...,8,0.803,Fort Wayne Kekiongas,Hamilton Field,NaN,101,107,KEK,FW1,FW1
4,1871,NaN,NY2,NNA,NaN,5,33,NaN,16,17,...,14,0.840,New York Mutuals,Union Grounds (Brooklyn),NaN,90,88,NYU,NY2,NY2
5,1871,NaN,PH1,PNA,NaN,1,28,NaN,21,7,...,13,0.845,Philadelphia Athletics,Jefferson Street Grounds,NaN,102,98,ATH,PH1,PH1
6,1871,NaN,RC1,ROK,NaN,9,25,NaN,4,21,...,14,0.821,Rockford Forest Citys,Agricultural Society Fair Grounds,NaN,97,99,ROK,RC1,RC1
7,1871,NaN,TRO,TRO,NaN,6,29,NaN,13,15,...,22,0.845,Troy Haymakers,Haymakers' Grounds,NaN,101,100,TRO,TRO,TRO
8,1871,NaN,WS3,OLY,NaN,4,32,NaN,15,15,...,20,0.850,Washington Olympics,Olympics Grounds,NaN,94,98,OLY,WS3,WS3
9,1872,NaN,BL1,BLC,NaN,2,58,NaN,35,19,...,22,0.830,Baltimore Canaries,Newington Park,NaN,106,102,BAL,BL1,BL1


In [13]:
Big_batters=(batting
    >> select(X.playerID,X.teamID,X.HR,X.yearID)
    #>> group_by(X.yearID)
    #>> mutate(Max_HR = X.HR.max())
    #>> filter_by(X.playerID == "bondsba01")
    >> filter_by(X.HR >50)
)
Big_batters

,playerID,teamID,HR,yearID
18534,ruthba01,NYA,54,1920
19054,ruthba01,NYA,59,1921
22271,ruthba01,NYA,60,1927
22801,ruthba01,NYA,54,1928
23980,wilsoha01,CHN,56,1930
24654,foxxji01,PHA,58,1932
27767,greenha01,DET,58,1938
33002,kinerra01,PIT,51,1947
33104,mizejo01,NY1,51,1947
34154,kinerra01,PIT,54,1949


## There are no players with over 100 runs in a season

In [29]:
intermediate=(Big_batters 
 >> left_join(people, by= "playerID")
 >> select(X.playerID, X.teamID, X.HR, X.nameLast, X.nameGiven,X.yearID))
 #>> inner_join(teams, by= "teamID")
 #>> select(X.playerID, X.teamID, X.HR, X.nameLast, X.nameGiven, X.name, X.W,X.L))
intermediate

,playerID,teamID,HR,nameLast,nameGiven,yearID
0,ruthba01,NYA,54,Ruth,George Herman,1920
1,ruthba01,NYA,59,Ruth,George Herman,1921
2,ruthba01,NYA,60,Ruth,George Herman,1927
3,ruthba01,NYA,54,Ruth,George Herman,1928
4,wilsoha01,CHN,56,Wilson,Lewis Robert,1930
5,foxxji01,PHA,58,Foxx,James Emory,1932
6,greenha01,DET,58,Greenberg,Henry Benjamin,1938
7,kinerra01,PIT,51,Kiner,Ralph McPherran,1947
8,mizejo01,NY1,51,Mize,John Robert,1947
9,kinerra01,PIT,54,Kiner,Ralph McPherran,1949


In [34]:
(intermediate 
    >> left_join(teams, by=["teamID","yearID"])
    >> select(X.playerID, X.teamID, X.HR_x, X.nameLast, X.nameGiven, X.name,X.yearID)
)


,playerID,teamID,HR_x,nameLast,nameGiven,name,yearID
0,ruthba01,NYA,54,Ruth,George Herman,New York Yankees,1920
1,ruthba01,NYA,59,Ruth,George Herman,New York Yankees,1921
2,ruthba01,NYA,60,Ruth,George Herman,New York Yankees,1927
3,ruthba01,NYA,54,Ruth,George Herman,New York Yankees,1928
4,wilsoha01,CHN,56,Wilson,Lewis Robert,Chicago Cubs,1930
5,foxxji01,PHA,58,Foxx,James Emory,Philadelphia Athletics,1932
6,greenha01,DET,58,Greenberg,Henry Benjamin,Detroit Tigers,1938
7,kinerra01,PIT,51,Kiner,Ralph McPherran,Pittsburgh Pirates,1947
8,mizejo01,NY1,51,Mize,John Robert,New York Giants,1947
9,kinerra01,PIT,54,Kiner,Ralph McPherran,Pittsburgh Pirates,1949
